<a href="https://colab.research.google.com/github/lucasnserra/Market-Basket-Analysis-in-Python/blob/main/2%C2%B0_Market_Basket_Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El metodo de agregación en Market Basket servira para agregar en distintas categorias los distintos productos ya sea por alguna palabra que contenga su titulo o caracteristica. Esto reduce el problema del análisis de la canasta de mercado a la identificación de reglas entre categorías de artículos. En nuestro caso, esto podría significar seleccionar el subconjunto de artículos que son bolsas y cajas y agregarlos en categorías de bolsas y cajas.

In [ ]:
#Importo Librerias
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import permutations
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
#Cargo dataset y convierto en lista 
gifts = pd.read_csv('https://raw.githubusercontent.com/lucasnserra/Market-Basket-Analysis-in-Python/main/online_retail.csv')
print(gifts.head(2))
#cantidad de facturas
Q = len(gifts['InvoiceNo'].unique())
#cantidad de Items
I = len(gifts['Description'].unique())

print('Cantidad de facturas unicas ', format(Q))
print('Cantidad de Items unicos ', format(I))


  InvoiceNo StockCode                      Description
0    562583    35637A  IVORY STRING CURTAIN WITH POLE 
1    562583    35638A    PINK AND BLACK STRING CURTAIN
Cantidad de facturas unicas  9709
Cantidad de Items unicos  3461


In [ ]:
#Preprocessing (OneHot) 
List = list(gifts['Description'].apply(str))
#Creo funcion para iterar entre las distintas filas y cambio previamente el formato ya que generaba error de int a str 
def extractDigits(List):
    return [[el] for el in List]
List = extractDigits(List)

In [ ]:
#Creo el encoder y lo entreno
encoder = TransactionEncoder().fit(List)
#Hago el OneHot (Crea una matriz de True o False si aparece el atributo en la fila)
onehot = encoder.transform(List)
#Creo DF 
onehot = pd.DataFrame(onehot,columns=encoder.columns_)
onehot.sample(2)

,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,...,wet boxes,wet pallet,wet rusty,wet?,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804
141998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
79614,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
#Agregacion en distintas categorias 
bag_headers = [i for i in onehot.columns if i.lower().find('bag')>=0]
box_headers = [i for i in onehot.columns if i.lower().find('box')>=0]
sign_headers = [i for i in onehot.columns if i.lower().find('sign')>=0]
#Identificamos y seleccionamos las columnas
bags = onehot[bag_headers]
boxes = onehot[box_headers]
sign = onehot[sign_headers]
#Sumo por columns 
bags = (bags.sum(axis=1) > 0.0).values
boxes = (boxes.sum(axis=1) > 0.0).values
sign = (sign.sum(axis=1) > 0.0).values
#Agrego el resultado a un DF
aggregated = pd.DataFrame(np.vstack([bags,boxes,sign]).T,columns = ['bags','boxes','Signs'])
aggregated.sample(10)

,bags,boxes,Signs
98621,False,False,False
42938,False,False,False
143991,False,False,False
17157,True,False,False
124568,True,False,False
152309,False,False,False
213946,False,False,False
74312,False,False,False
94019,False,False,False
138722,False,False,False


In [ ]:
#Aplicacion de metricas (ejemplo soporte)
aggregated.mean()

bags     0.094569
boxes    0.051567
Signs    0.081915
dtype: float64

In [ ]:
#automatizacion de los pasos anteriores
def aggregate(item):
  item_headers = [i for i in onehot.columns if i.lower().find(item) >= 0 ]
  item_columns = onehot[item_headers]
  return item_columns.sum(axis=1) >= 1.0

#Pruebo la funcion
bags = aggregate('bag')
boxes = aggregate('boxes')
candles = aggregate('candles')

aggregated = pd.DataFrame(np.vstack([bags,boxes,candles]).T,columns = ['bags','boxes','candles'])
aggregated.mean()

bags       0.094569
boxes      0.004676
candles    0.010256
dtype: float64